**CSE 5522 Lab #2: Sentiment Analysis**

The goals of this lab are to familarize you with:

*   Naive Bayes
*   Binary Classification
*   Data exploration
*   Working with text-based data (Tweets)

**Initial notes**

* (If you are using Google Colab) Make a copy of this page in your google drive so that you can edit it.

* While not completely necessary for this assignment, you may want to familiarize yourself with the following packages: [numpy](https://numpy.org), [scikit-learn](https://scikit-learn.org), [pandas](https://pandas.pydata.org), [matplotlib](https://matplotlib.org).
 * Especially numpy, many of the calculations in this (and later) lab can be done in one line using numpy. Whereas raw python may require 5-10x that.

* Feel free to (please do!) change the structure of the document below. Especially, add code sections to break your code into logical pieces and add text sections to explain your code or results

---
---

**Part 1: Hands-On #2 (0 pts)**

You will need to finish any remaining parts of the Hands-On that you didn't finish in class. Feel free to use code from the posted Hands-On #2 (partial) solution.

Note, you do not need to explicitly include the your Hands-On #2 solution here. But parts 2 and 3 assume you have a working solution, and you will probably need/want to copy code from it when you work on them.

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

TweetUrl='https://github.com/aasiaeet/cse5522data/raw/master/db3_final_clean.csv'
tweet_dataframe=pd.read_csv(TweetUrl)

# wordDict maps words to id
# X is the document-word matrix holding the presence/absence of words in each tweet
wordDict = {}
idCounter = 0
for i in range(tweet_dataframe.shape[0]):
  allWords = tweet_dataframe.iloc[i,1].split(" ")
  for word in allWords:
    if word not in wordDict:
      wordDict[word] = idCounter
      idCounter += 1
X = np.zeros((tweet_dataframe.shape[0], idCounter),dtype='float')
for i in range(tweet_dataframe.shape[0]):
  allWords = tweet_dataframe.iloc[i,1].split(" ")
  for word in allWords:
    X[i, wordDict[word]]  = 1

y = np.array(tweet_dataframe.iloc[:,2])

---
---

**Part 2: An Alternate Model (50 pts)**

In Part 1, you calculated the probability of a tweet by incorporating both the probability of words present in the tweet $P\left(x^i_j=1 | +\right)$ and the probability of words absent from the tweet $P\left(x^i_j=0 | +\right)$.

Now, modify your code to *only* incorporate the probability of words present in the tweet $P\left(x^i_j=1 | +\right)$ (thus ignoring absent words).

Compare this to the original approach in Part 1. Follow reasonable experimental procedure and write up an explanation of the results you find.


In [3]:
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 0)

# borrowed from hands-on partialsolutions
def compute_distros(x,y):
  # probWordGivenPositive: P(word|Sentiment = +ive)
  probWordGivenPositive=np.mean(x[y>=0,:],axis=0) #Sum each word (column) to count how many times each word shows up (in positive examples)
                                                  #and Divide by total number of (positive) examples to give distribution

  # probWordGivenNegative: P(word|Sentiment = -ive)
  probWordGivenNegative=np.mean(x[y<0,:],axis=0)

  # priorPositive: P(Sentiment = +ive)
  priorPositive = np.mean(y>=0) #Number of positive examples vs. all examples
  # priorNegative: P(Sentiment = -ive)
  priorNegative = 1 - priorPositive
  #  (note these last two form one distribution)
  return probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative

# compute distributions here
probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative = compute_distros(xTrain,yTrain)

# checking the results
display(probWordGivenPositive[0:5])
display(probWordGivenNegative[0:5])
display(priorPositive, priorNegative)

array([0.1185006 , 0.20737606, 0.01088271, 0.01451028, 0.10217654])

array([0.14504988, 0.19493477, 0.00537222, 0.09669992, 0.13967767])

0.5593506932702063

0.44064930672979374

In [4]:
# from hands-on solutions again

def compute_logdistros(distros, min_prob):
  if True:
    #Assume missing words are simply very rare
    #So, assign minimum probability to very small elements (e.g. 0 elements)
    distros=np.where(distros>=min_prob,distros,min_prob)
    #Also need to consider minimum probability for "not" distribution
    distros=np.where(distros<=(1-min_prob),distros,1-min_prob)

    return np.log(distros), np.log(1-distros)
  else:
    #Ignore missing words (assume they have P==1, i.e. force log 0 to 0)
    return np.log(np.where(distros>0,distros,1)), np.log(np.where(distros<1,1-distros,1))

min_prob = 1/yTrain.shape[0] #Assume very rare words only appeared once
# absent probabilities are unused here
logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive = compute_logdistros(probWordGivenPositive,min_prob) 
logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative = compute_logdistros(probWordGivenNegative,min_prob)
logPriorPositive, logPriorNegative = compute_logdistros(priorPositive,min_prob)

# Did this work, or did you get an error?  (Read below.)
display(logProbWordPresentGivenPositive[0:5])
display(logProbWordPresentGivenNegative[0:5])
display(logPriorPositive, logPriorNegative)

array([-2.13283722, -1.57322143, -4.52058012, -4.23289805, -2.28105316])

array([-1.93067756, -1.63509031, -5.22651443, -2.33614267, -1.96841789])

-0.5809786442688406

-0.819505942727632

In [5]:
def classifyNB(words,logProbWordPresentGivenPositive, 
               logProbWordPresentGivenNegative, 
               logPriorPositive, logPriorNegative):
  positiveProb = logPriorPositive
  negativeProb = logPriorNegative
  for i in range(len(words)):
    if i < len(words)-1:
      nextPresent = (words[i+1] == 1)
    #if words[i] == 0 and i % 10 == 9 and nextPresent:
      #print(list(wordDict.items())[i])
      #print("absent, positiveProb = {}, negativeProb = {}".format(positiveProb, negativeProb))
    if words[i] == 1:
      positiveProb += logProbWordPresentGivenPositive[i]
      negativeProb += logProbWordPresentGivenNegative[i]
      #print(list(wordDict.items())[i])
      #if i % 10 == 0 or nextPresent:
        #print(list(wordDict.items())[i])
        #print("present, positiveProb = {}, negativeProb = {}".format(positiveProb, negativeProb))
  if positiveProb > negativeProb:
    label = "positive"
    confidence = positiveProb - negativeProb
  else:
    label = "negative"
    confidence = negativeProb - positiveProb
  return [label, confidence]
  
for j in range(100):
  print()
  print(tweet_dataframe.iloc[j,1])
  print(classifyNB(xTest[j, ], logProbWordPresentGivenPositive,
                                 logProbWordPresentGivenNegative,
                                 logPriorPositive, logPriorNegative))


it is very cold out want it to be warmer 
['negative', 3.7020748809533615]

dammmmmmm its pretty cold this morning   burr lol
['positive', 1.1523083255317488]

why does halsey have to be so far away think my ears are ready to fall off cold
['negative', 4.891369578349]

dammit stop being so cold so can work out 
['positive', 3.4535121858089894]

its too freakin cold
['positive', 5.514477107118434]

i  freezing my tits off hate the cold
['negative', 2.1280352589918152]

 its too cold to go out sad 
['negative', 2.9315461853343265]

its hate minnesota
['positive', 1.1160302455781732]

ok exactly what im thinking rt want to go out but dont want to deal with the cold weather smh
['negative', 1.4731182164487393]

 girl think the same shit too fucking cold out 
['negative', 4.591765958903238]

hello its cold as b tch out wtf 
['positive', 10.346318295815507]

it is to dang cold to do anything out tonight stayinhometweet
['positive', 4.94914098437971]

sitting outside because the hose just go

In [6]:
# tests NB and outputs accuracy much like in the hands-on, but without the absent parameters
def testNB(xTest, yTest, 
           logProbWordPresentGivenPositive, 
           logProbWordPresentGivenNegative, 
           logPriorPositive, logPriorNegative):
  
  accuracy = 0
  for i in range(xTest.shape[0]):
    testLabelText = classifyNB(xTest[i, ], logProbWordPresentGivenPositive,
                                 logProbWordPresentGivenNegative,
                                 logPriorPositive, logPriorNegative)[0]
    testLabelNum = -1
    if testLabelText == "positive":
      testLabelNum = 1
    if testLabelNum == yTest[i]:
      accuracy += 1
  
  accuracy /= xTest.shape[0]
  print("Accuracy of NB is", accuracy)
  return accuracy

accuracies = []
for i in range(50):
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2)
  probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative = compute_distros(xTrain,yTrain)
  min_prob = 1/yTrain.shape[0] #Assume very rare words only appeared once
  logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive = compute_logdistros(probWordGivenPositive,min_prob)
  logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative = compute_logdistros(probWordGivenNegative,min_prob)
  logPriorPositive, logPriorNegative = compute_logdistros(priorPositive,min_prob)
  accuracies.append(testNB(xTest, yTest, 
       logProbWordPresentGivenPositive,
       logProbWordPresentGivenNegative,
       logPriorPositive, logPriorNegative))
print("Mean accuracy: {}".format(np.mean(accuracies)))
print("Standard deviation of accuracy: {}".format(np.std(accuracies)))

Accuracy of NB is 0.8297297297297297
Accuracy of NB is 0.8283783783783784
Accuracy of NB is 0.8432432432432433
Accuracy of NB is 0.8121621621621622
Accuracy of NB is 0.8445945945945946
Accuracy of NB is 0.8121621621621622
Accuracy of NB is 0.8054054054054054
Accuracy of NB is 0.8337837837837838
Accuracy of NB is 0.8162162162162162
Accuracy of NB is 0.827027027027027
Accuracy of NB is 0.8108108108108109
Accuracy of NB is 0.8148648648648649
Accuracy of NB is 0.831081081081081
Accuracy of NB is 0.8297297297297297
Accuracy of NB is 0.8391891891891892
Accuracy of NB is 0.8243243243243243
Accuracy of NB is 0.8121621621621622
Accuracy of NB is 0.8135135135135135
Accuracy of NB is 0.8472972972972973
Accuracy of NB is 0.8432432432432433
Accuracy of NB is 0.85
Accuracy of NB is 0.8337837837837838
Accuracy of NB is 0.8175675675675675
Accuracy of NB is 0.8216216216216217
Accuracy of NB is 0.8351351351351352
Accuracy of NB is 0.8202702702702702
Accuracy of NB is 0.8432432432432433
Accuracy of NB is

**Part 1 (50 runs):**

Mean accuracy: 0.82462

Standard deviation of accuracy: 0.012603

**Part 2 (50 runs):**

Mean accuracy: 0.82868

Standard deviation of accuracy: 0.012190

Part 2's results were near identical to Part 1's, sometimes with slightly higher mean or standard deviation, sometimes slightly lower. This lack of difference suggests that absent probabilities had little to no bearing on how tweets were predicted. This makes sense because while most present probabilities were close to 0, most absent probabilities were close to 1 and thus did not affect the prediction (derived from a product of probabilities) very much.

---
---
**Part 3: An Additional Experiment (50 pts)**

Implement an experiment to change the model, and report on the results of the experiment, comparing to your baseline model from Part 1 (and your alternate model from Part 2).

**3.3: Option 3: Sticky terms (6 bonus points for difficulty)**

A "sticky term" is two words which are more likely to occur together than independently.

You can use "Pointwise Mutual Information" (PMI) to determine the stickiness, using: $PMI=\frac{P(w_1,w_2)}{P(w_1)P(w_2)}$.  For all pairs of <u>adjacent</u> words in the tweet corpus, find the top n pairs according to PMI and add them as additional features in your Naive Bayes Model.

Find the top 100, 200, 500 "sticky terms" and add these as features to the model.

(Note, you cannot use X to calculate the above joint distribution, since the above is about adjacent words. You will have to go back to the raw text.)

---

Remember, you need to *compare* your chosen option with your previous work. Just writing the code is not sufficient. Follow reasonable experimental procedure and write up a discussion of your results and why you think they turned out that way.

In [ ]:
from collections import Counter # used for the top 100 part

wordDict = {}
idCounter = 0
wordProbs = {} # new to this portion
# sticky variables are new to this portion
stickyDict = {}
stickyIdToWordPairs = []
stickyIdCounter = 0
stickyProbs = {}
stickyPMIs = {}

top = 500 # change this to 200 or 500 to test further

for i in range(tweet_dataframe.shape[0]):
  allWords = tweet_dataframe.iloc[i,1].split(" ")
  for j in range(len(allWords)):
    word = allWords[j]
    if word not in wordDict:
      wordDict[word] = idCounter
      idCounter += 1
      wordProbs[wordDict[word]] = 1 / tweet_dataframe.shape[0]
    else:
      wordProbs[wordDict[word]] += 1 / tweet_dataframe.shape[0]
    if j > 0:
      prevWord = allWords[j-1]
      if prevWord != "" and word != "":
        wordPair = str(wordDict[prevWord]) + ", " + str(wordDict[word])
        if wordPair not in stickyDict:
          stickyDict[wordPair] = stickyIdCounter
          stickyIdCounter += 1
          stickyIdToWordPairs.append(prevWord + " " + word)
          stickyProbs[wordPair] = 1 / tweet_dataframe.shape[0]
        else:
          stickyProbs[wordPair] += 1 / tweet_dataframe.shape[0]

for wordPair in stickyProbs:
  words = wordPair.split(", ")
  # freqThreshold is used so that pairs must consist of two words that each occur at least 10 times in the tweets.
  # I put the variable in because otherwise, the top 100 sticky terms would all be pairs of words that each occur only once.
  freqThreshold = 10 / tweet_dataframe.shape[0]
  if wordProbs[int(words[0])] >= freqThreshold and wordProbs[int(words[1])] > freqThreshold:
    pmi = stickyProbs[wordPair]/(wordProbs[int(words[0])] * wordProbs[int(words[1])])
    # print("{}, PMI {}".format(stickyIdToWordPairs[stickyDict[wordPair]], pmi))
    stickyPMIs[stickyDict[wordPair]] = pmi

    #if j == 10:
      #print("{}, ID {}, frequency {}".format(word, wordDict[word], wordFreqs[wordDict[word]]))
      #print("{} {}, ID {}, frequency {}".format(prevWord, word, stickyDict[wordPair], stickyFreqs[stickyDict[wordPair]]))

stickyTopWithPMIs = Counter(stickyPMIs).most_common(top)
stickyTop = []
for pair in stickyTopWithPMIs:
  stickyTop.append(pair[0])
  print("{}, PMI {}".format(stickyIdToWordPairs[pair[0]], pair[1]))

# a new modified version of X is created, with 100 more features than before
# the new 100 features are the 100 most common sticky terms, sorted by frequency

X = np.zeros((tweet_dataframe.shape[0], idCounter + top),dtype='float')
for i in range(tweet_dataframe.shape[0]):
  allWords = tweet_dataframe.iloc[i,1].split(" ")
  for j in range(len(allWords)):
    word = allWords[j]
    X[i, wordDict[word]] = 1
    if j > 0:
      prevWord = allWords[j-1]
      if prevWord != "" and word != "":
        wordPair = str(wordDict[prevWord]) + ", " + str(wordDict[word])
        pairNum = stickyDict[wordPair]
        #print(stickyDict[wordPair])
        #print(stickyTop[1])
        if stickyDict[wordPair] in stickyTop:
          rank = stickyTop.index(stickyDict[wordPair])
          X[i, (len(wordDict) + rank)] = 1
          #print("{} {}, ID {}, rank {}".format(prevWord, word, stickyDict[wordPair], rank))
  #print(X[i, len(wordDict):(len(wordDict) + top)])
          

# on the other hand, y (the judgements of the tweets) is the same as ever

y = np.array(tweet_dataframe.iloc[:,2])

mother nature, PMI 166.8784722222223
egg hunt, PMI 133.94927536231893
st louis, PMI 119.25806451612907
couldnt ask, PMI 87.6758893280633
blue skies, PMI 82.15555555555562
blue sky, PMI 65.72444444444449
golf course, PMI 60.93956043956048
bike ride, PMI 60.4963636363637
wait till, PMI 59.62903225806455
tornado warning, PMI 57.4068322981367
clear blue, PMI 56.87692307692311
clean room, PMI 56.015151515151544
light show, PMI 47.397435897435926
light breeze, PMI 47.397435897435926
dear mother, PMI 45.64197530864202
along lake, PMI 44.81212121212124
watch movies, PMI 40.62637362637366
mile run, PMI 40.53728070175438
didnt show, PMI 38.51041666666669
hello world, PMI 37.91794871794875
decent thought, PMI 36.24509803921571
stay safe, PMI 36.2450980392157
eggs despite, PMI 35.54807692307695
news said, PMI 35.20952380952384
an hour, PMI 33.69661458333333
clean air, PMI 33.60909090909094
gets warmer, PMI 33.45701357466067
april showers, PMI 32.50109890109891
clear skies, PMI 31.59829059829062
fe

In [ ]:
min_prob = 1/yTrain.shape[0] #Assume very rare words only appeared once
# absent probabilities are unused here
logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive = compute_logdistros(probWordGivenPositive,min_prob) 
logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative = compute_logdistros(probWordGivenNegative,min_prob)
logPriorPositive, logPriorNegative = compute_logdistros(priorPositive,min_prob)

# Did this work, or did you get an error?  (Read below.)
display(logProbWordPresentGivenPositive[0:5])
display(logProbWordPresentGivenNegative[0:5])
display(logPriorPositive, logPriorNegative)

for j in range(100):
  print()
  print(tweet_dataframe.iloc[j,1])
  print(classifyNB(xTest[j, ], logProbWordPresentGivenPositive,
                                 logProbWordPresentGivenNegative,
                                 logPriorPositive, logPriorNegative))

array([-2.13515215, -1.58450503, -4.82769816, -4.21459369, -2.29278109])

array([-1.99669029, -1.68579898, -5.11574578, -2.39939677, -2.05352376])

-0.5992829991308539

-0.7967431996737686


it is very cold out want it to be warmer 
['negative', 12.534095398235976]

dammmmmmm its pretty cold this morning   burr lol
['negative', 2.6282775053518517]

why does halsey have to be so far away think my ears are ready to fall off cold
['positive', 1.5019285662463915]

dammit stop being so cold so can work out 
['positive', 0.6166277233077864]

its too freakin cold
['positive', 2.902144562806585]

i  freezing my tits off hate the cold
['positive', 9.485594848716431]

 its too cold to go out sad 
['positive', 3.472173786132032]

its hate minnesota
['negative', 2.144127501987896]

ok exactly what im thinking rt want to go out but dont want to deal with the cold weather smh
['negative', 1.4809102910057703]

 girl think the same shit too fucking cold out 
['negative', 4.074543305813641]

hello its cold as b tch out wtf 
['positive', 5.262213018266671]

it is to dang cold to do anything out tonight stayinhometweet
['positive', 5.774593507200734]

sitting outside because the hose just g

In [ ]:
accuracies = []
for i in range(50):
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2)
  probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative = compute_distros(xTrain,yTrain)
  min_prob = 1/yTrain.shape[0] #Assume very rare words only appeared once
  logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive = compute_logdistros(probWordGivenPositive,min_prob)
  logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative = compute_logdistros(probWordGivenNegative,min_prob)
  logPriorPositive, logPriorNegative = compute_logdistros(priorPositive,min_prob)
  accuracies.append(testNB(xTest, yTest, 
       logProbWordPresentGivenPositive,
       logProbWordPresentGivenNegative,
       logPriorPositive, logPriorNegative))
print("Mean accuracy: {}".format(np.mean(accuracies)))
print("Standard deviation of accuracy: {}".format(np.std(accuracies)))

Accuracy of NB is 0.8202702702702702
Accuracy of NB is 0.8189189189189189
Accuracy of NB is 0.8337837837837838
Accuracy of NB is 0.8189189189189189
Accuracy of NB is 0.845945945945946
Accuracy of NB is 0.8013513513513514
Accuracy of NB is 0.8189189189189189
Accuracy of NB is 0.8040540540540541
Accuracy of NB is 0.8148648648648649
Accuracy of NB is 0.8364864864864865
Accuracy of NB is 0.8135135135135135
Accuracy of NB is 0.822972972972973
Accuracy of NB is 0.7945945945945946
Accuracy of NB is 0.8202702702702702
Accuracy of NB is 0.8364864864864865
Accuracy of NB is 0.8189189189189189
Accuracy of NB is 0.8351351351351352
Accuracy of NB is 0.8405405405405405
Accuracy of NB is 0.8202702702702702
Accuracy of NB is 0.8445945945945946
Accuracy of NB is 0.8297297297297297
Accuracy of NB is 0.8175675675675675
Accuracy of NB is 0.8202702702702702
Accuracy of NB is 0.8337837837837838
Accuracy of NB is 0.8094594594594594
Accuracy of NB is 0.8337837837837838
Accuracy of NB is 0.8337837837837838
Acc

**Part 3 (50 runs, top 100 pairs):**

Mean accuracy: 0.82724

Standard deviation of accuracy: 0.013233

**Part 3 (50 runs, top 200 pairs):**

Mean accuracy: 0.82568

Standard deviation of accuracy: 0.012962

**Part 3 (50 runs, top 500 pairs):**

Mean accuracy: 0.82372

Standard deviation of accuracy: 0.012509

From these results, the addition of sticky terms changed very little. There is a slight decrease in mean accuracy and standard deviation of accuracy corresponding to increase in number of top pairs added, which is likely to be a coincidence.

I think the reason for this lack of change is that the single words far outnumber the sticky terms, with almost 6000 single words and only up to 500 sticky terms. This means that the sticky terms probably weren't able to leave a significant change to the results, especially considering that the tweets more often than not lacked sticky terms.